In [2]:
print("hola mundo")

hola mundo


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.getActiveSession()
if spark is None:
    spark = SparkSession.builder.appName("MiApp").getOrCreate()
    
spark.range(1_000_000).count()



1000000

In [4]:
spark.sql("""
    select *
    from comunes_canales_dev.gold.intermediarios
    limit 10
""").show()

+----------------------+--------------------+---------------------+--------------------+----------------------+------------------+--------------------+---------------+--------------------+-------------------+--------------------+---------------+--------------------+
|Codigo Oficial Negocio|     Oficial Negocio|Codigo Grupo Cabecera|      Grupo Cabecera|Codigo Grupo Cta. Cte.|Codigo Organizador|         Organizador|Codigo Producor|            Producor|Codigo Participante|        Participante|Codigo Compania|     Intermediary_PK|
+----------------------+--------------------+---------------------+--------------------+----------------------+------------------+--------------------+---------------+--------------------+-------------------+--------------------+---------------+--------------------+
|                   178|    MANCIOLI MARCELO|               100440|   MERONI RAUL OSCAR|        D0000013823236|            100440|   MERONI RAUL OSCAR|         270735|      MERONI JOAQUIN|           

In [ ]:
%pip install --upgrade --force-reinstall './notebooks/gss_bi_udfs-0.1.1-py3-none-any.whl'

In [ ]:
from gss_bi_udfs.io import load_and_materialize_views

In [ ]:
tables_load = {
    "timepro": {
        "insudb": [
            {"table": "fire", "view": "vw_fire"},
            {"table": "table7045", "view": "vw_table7045"},
            {"table": "table233", "view": "vw_table233"},
            {"table": "table118", "view": "vw_table118"},
            {"table": "table7046", "view": "vw_table7046"},
            {"table": "table7037", "view": "vw_table7037"},
            {"table": "table7038", "view": "vw_table7038"},  
            {"table": "table7039", "view": "vw_table7039"}             
        ]
    }
 }    

params = {"spark":spark, "tables_load":tables_load, "verbose":True, "env":"dev"}
dfs = load_and_materialize_views('parquets_register_temp_views', **params)

# otra version para escribir un solo archivo .parquet

In [ ]:
from pyspark.sql import SparkSession
import time
from os.path import dirname, basename

from pyspark.sql.functions import rand, col

# Create a random DataFrame
df_random = spark.range(1000).select(
    col("id"),
    (rand() * 100).cast("int").alias("random_value1"),
    (rand() * 1000).cast("int").alias("random_value2")
)

spark = SparkSession.builder.getOrCreate()

timestamp = time.strftime("%Y%m%d_%H%M%S")

base_path = "/Volumes/bronze/timepro_insudb/dev/table233"
tmp_path = f"{base_path}/_tmp"
final_file = f"{base_path}/table233_{timestamp}.parquet"

# 1️⃣ escribir en carpeta temporal
(
    df_random
    .repartition(1)
    .write
    .mode("overwrite")
    .parquet(tmp_path)
)

# 2️⃣ Hadoop FileSystem
hadoop_conf = spark._jsc.hadoopConfiguration()
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(hadoop_conf)

tmp_path_hadoop = spark._jvm.org.apache.hadoop.fs.Path(tmp_path)
final_path_hadoop = spark._jvm.org.apache.hadoop.fs.Path(final_file)

# 3️⃣ buscar el part-*.parquet
for file_status in fs.listStatus(tmp_path_hadoop):
    name = file_status.getPath().getName()
    if name.startswith("part-") and name.endswith(".parquet"):
        part_path = file_status.getPath()
        break

# 4️⃣ mover y renombrar
fs.rename(part_path, final_path_hadoop)

# 5️⃣ borrar carpeta temporal (_SUCCESS, etc.)
fs.delete(tmp_path_hadoop, True)

# 6️⃣ borrar crc
final_dir = dirname(final_file)
final_name = basename(final_file)
crc_file = f"{final_dir}/.{final_name}.crc"
crc_path = spark._jvm.org.apache.hadoop.fs.Path(crc_file)

if fs.exists(crc_path):
    fs.delete(crc_path, False)


In [ ]:
from gss_bi_udfs.io import _normalize_path, _ls_path
from pathlib import Path

In [ ]:
from pathlib import Path

base_path = Path(f"Volumes/bronze/timepro_insudb/dev/table233")
print(base_path.iterdir())
[print (f"{str(child)}") for child in base_path.iterdir()]

In [ ]:
base_path = f"/Volumes/bronze/timepro_insudb/dev/table233"
# _normalize_path(base_path)
files = _ls_path(base_path)
table = "table233"
parquet_files = [f for f in files if table in f.name]

In [ ]:
base_path

In [ ]:
for file in parquet_files:
    print(file.name)

In [ ]:
latest_file = sorted(parquet_files, key=lambda f: f.name, reverse=True)[0]
latest_file.path
# df = spark.read.parquet(latest_file.path)

In [ ]:
from gss_bi_udfs.io import load_latest_parquet

df = load_latest_parquet(spark, "timepro", "insudb", "table233", env="dev")


In [ ]:
df.head()


# Mega orquestador

In [5]:
from gss_spark_flow.context import PipelineContext

In [ ]:
spark.sql(f"""
        CREATE TABLE IF NOT EXISTS operaciones_dev.metadata.data_quality (
            run_id STRING,
            task_name STRING,
            check_name STRING,
            metric_value DOUBLE,
            threshold_value DOUBLE,
            status STRING,
            created_ts TIMESTAMP
        )
        USING DELTA
""")

In [6]:
from gss_spark_flow.init_metadata import init_metadata_layer
init_metadata_layer(spark, catalog="operaciones_dev", schema="metadata")

Creating schema with SQL: CREATE SCHEMA IF NOT EXISTS operaciones_dev.metadata


{"ts": "2026-02-27 10:31:52.201", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[TABLE_OR_VIEW_NOT_FOUND] The table or view `operaciones_dev`.`metadata`.`run` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 1 pos 9;\n'DescribeRelation false, [col_name#256, data_type#257, comment#258]\n+- 'UnresolvedTableOrView [operaciones_dev, metadata, run], DESCRIBE TABLE, true, true\n\n\nJVM stacktrace:\norg.apache.spark.sql.catalyst.ExtendedAnalysisException\n\tat org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.tableNotFound(package.scala:94)\n\tat org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2(CheckAnalysis.scala:326)\n\tat org.apache.spark.sql.catalyst.analysis.Che

Metadata layer initialized. Version: 1.0.0
